In [ ]:
pip install spade

# Ejemplo de subasta inglesa: se propone un precio inicial y los bidders van aumentando sus pujas hasta un maximo propio

**Recuerda cambiar el nombre de los agentes**

In [ ]:
import spade
from spade.behaviour import CyclicBehaviour, OneShotBehaviour
from spade.message import Message

class AuctioneerAgent(spade.agent.Agent):
    def __init__(self, jid, password, bidders):
        super().__init__(jid, password)
        self.bidders = bidders  # Lista dinámica de pujadores

    
    class AuctionBehaviour(OneShotBehaviour):
        async def run(self):
            # Iniciar la subasta con un precio base
            current_price = 100  # Precio inicial
            winner = None
            print(f"Subastador envía el precio inicial {current_price}")
            while self.agent.bidders:
                # Enviar el precio actual a todos los pujadores
                for bidder in self.agent.bidders:
                    msg = Message(to=bidder)
                    msg.body = str(current_price)
                    #print(f"Subastador envía a {bidder}: Precio actual {current_price}")
                    await self.send(msg)


                await asyncio.sleep(1)
                
                # Recibir ofertas o mensajes de retiro
                highest_bid = current_price
                for _ in range(len(self.agent.bidders)):
                    msg = await self.receive(timeout=3)
                    if msg:
                        if msg.metadata["performative"] == "withdraw":
                            #print(f"{msg.sender} se ha retirado de la puja")
                            self.agent.bidders.remove(str(msg.sender))
                        else:
                            bid = int(msg.body)
                            #print(f"Subastador recibe de {msg.sender}: Oferta {bid}")
                            if bid > highest_bid:
                                highest_bid = bid
                                winner = msg.sender

                # Comprobar si el precio ha aumentado
                if (highest_bid == current_price or len(self.agent.bidders) == 1):
                    break  # Finalizar la subasta si no hay nuevas ofertas o solo queda 1
                
                current_price = highest_bid

            # Anunciar el ganador
            print(f"El ganador es {winner} con una oferta de {current_price}")

    async def setup(self):
        self.add_behaviour(self.AuctionBehaviour())


In [ ]:
import random

class BidderAgent(spade.agent.Agent):
    def __init__(self, jid, password):
        super().__init__(jid, password)
        self.max_price = random.randint(200, 300)  # Precio máximo aleatorio
        self.current_bid = 100  # Oferta inicial

    class BidBehaviour(CyclicBehaviour):
        async def run(self):
            msg = await self.receive(timeout=10)
            if msg:
                current_price = int(msg.body)
                #print(f"{self.agent.name} recibe: Precio actual {current_price}")
                if current_price < self.agent.max_price:
                    increment_factor = random.uniform(1.05, 1.15)  # Incremento aleatorio entre 5% y 15%
                    new_bid = min(self.agent.max_price, int(current_price * increment_factor))   # no nos pasamos
                    self.agent.current_bid = new_bid
                    reply = msg.make_reply()
                    reply.set_metadata("performative", "bid")
                    reply.body = str(self.agent.current_bid)
                    print(f"{self.agent.name} envía: Nueva oferta {self.agent.current_bid}")
                    await self.send(reply)
                else:
                    # Enviar mensaje de retiro
                    print(f"{self.agent.name} se retira de la puja")
                    withdraw_msg = msg.make_reply()
                    withdraw_msg.set_metadata("performative", "withdraw")
                    await self.send(withdraw_msg)
                    return
                    
    async def setup(self):
        self.add_behaviour(self.BidBehaviour())


In [ ]:
import asyncio

num_bidders = 10  # Número de pujadores

bidder_jids = [f"bidder{i}vjulian@gtirouter.dsic.upv.es" for i in range(1, num_bidders + 1)]
bidders = [BidderAgent(jid, "secret") for jid in bidder_jids]
for bidder in bidders:
    await bidder.start()

auctioneer = AuctioneerAgent("auctioneer_vjulian@gtirouter.dsic.upv.es", "secret", bidder_jids)
await auctioneer.start()



await asyncio.sleep(60)  # Duración de la subasta

await auctioneer.stop()
for bidder in bidders:
    await bidder.stop()


